In [ ]:
import urllib
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score 
from datetime import datetime, timedelta

## Teams Communication log

In [ ]:
TeamsLogUrl=("https://www.dropbox.com/s/tk9q2z34e8nyiqq/Teams%20Communication%20log.txt?dl=1")
file = urllib.request.urlopen(TeamsLogUrl)

In [ ]:
for line in file:
    decoded_line = line.decode("utf-8")
    print(decoded_line)

# Section 1

Authour : Ignas Rocas, 

Student Nr: C00135830,

Purpose : CA1 Data Science ,

### Clean data

In [ ]:
#Clean data
e_data = pd.read_csv("https://www.dropbox.com/s/ey2y94hyfclgl98/clean_data.csv?dl=1")
e_data.head()

In [ ]:
e_data.columns

In [ ]:
e_data['Year'] = e_data['Year'].apply(int)
e_data['Day'] = e_data['Day'].apply(int)

In [ ]:

start = datetime(1965,1,1)
current = datetime(e_data['Year'][0], e_data['Month'][0], e_data['Day'][0], 0, 0)
s = int((current - start).total_seconds())

In [ ]:
dates = list()
for i in range(len(e_data['Year'])):
    current = datetime(e_data['Year'][i], e_data['Month'][i], e_data['Day'][i], 0, 0)
    s = int((current - start).total_seconds())
    dates.append(s)
e_data['Date']=pd.Series(dates)
e_data.head()

In [ ]:
e_data['DATE'] = pd.to_datetime(e_data[['Year', 'Month','Day']])
e_data = e_data.drop(['Unnamed: 0','Month','Day','Year'],axis=1)
e_data.head()

# Corroletion matrix

In [ ]:
# Shows that the correletion bettween features are too great for Linear and Multi-Linear Regression
corrMatrix = e_data.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:

y = e_data['Latitude'].values
X = e_data['Longitude'].values.reshape(-1,1)

In [ ]:
# just to see how the points look and clearly n
plt.scatter(X, y, color='black')
plt.title('all the earthquakes')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Random forest

In [ ]:
#separate data & split data
X = e_data[['Longitude', 'Latitude', 'Magnitude','Density']]
y = e_data['Date']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
#user random forest
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state = 42)
forest.fit(X_train, y_train)
pred = forest.predict(X_test)

In [ ]:
#random forest evaluation
print("Score : ", forest.score(X_test,y_test))
print("Mean squared error : ", mean_squared_error(y_test,pred))

# DBSCAN Clustering (knns,decision trees)

In [ ]:
df = e_data
df

In [ ]:
#https://medium.com/@tarammullin/dbscan-parameter-estimation-ff8330e3a3bd
#followed article above which parameters to pick for DBSCAN

#used The k-nearest neighbors to pick the best eps for DBSCAN
neighbors = NearestNeighbors(n_neighbors=4)#neigbors set to min_samples
neighbors_fit = neighbors.fit(df[['Latitude','Longitude']])
distances, indices = neighbors_fit.kneighbors(df[['Latitude','Longitude']])
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.figure(figsize=(30,30))
plt.plot(distances)
# show that best eps is close to 2

In [ ]:
#user DBSCAN and save its labels to dataframe
dbscan=DBSCAN(eps=1.6,min_samples=4)
dbscan.fit_predict(df[['Longitude','Latitude']])
df['Cluster']=dbscan.labels_ 

In [ ]:
#plot all the data
def plot(df):
    plt.scatter(df['Longitude'],df['Latitude'],c=df['Cluster'],cmap="viridis_r")
    plt.title('DBSCAN Clustering',fontsize=20)
    plt.xlabel('Longitude',fontsize=14)
    plt.ylabel('Latitude',fontsize=14)
    plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plot(df)

In [ ]:
#get unique clusters and remove outliners
#(-1 lable,and any clusters with 1 element) - display the results

clusters = df['Cluster'].value_counts().index
clusters = list(clusters)
clusters.sort()
removeList = [-1,126]#126 has the same date in same cluster/ same location
temp =  [i for i in clusters if len(df[df.Cluster==i])<4]
removeList.extend(temp)

for i in removeList:
    if i in clusters:
        clusters.remove(i)
    
    
df = df[df['Cluster'].isin(clusters) == True]
plt.figure(figsize=(20,10))
plot(df)

In [ ]:
df.head()

In [ ]:
#get one cluster and display it
two = df[df.Cluster == 2]
plt.scatter(two['Longitude'],two['Latitude'],c=two['Cluster'],cmap="viridis_r")
plt.legend(two['Cluster'])
plt.show()

NOTES: So we get all the averages time in between the earthquakes for each of the cluster. Then we use Decetion trees to predict incomping new data (users location(longatude/latidue) => y(Cluster)) So we can derive the next earthquake data,from (previous know erthquake date)+(predicted average inbetween for that Cluster)

In [ ]:
df[df.Cluster == 2].Cluster.iloc[0]

In [ ]:
#calculate for each cluster the averate time inbetween the earthquakes 
avgCluDict = {}
for n in clusters:# --------all the unique cluster
    curCluster = df[df.Cluster == n]#-------------get each cluster
    curNumber = curCluster.Cluster.iloc[0] # get cluster number
    total=0
    count=0
    for x in range(len(curCluster)):
        a = curCluster['DATE'].iloc[x]
        x+=1
        if(x<len(curCluster)):
            b = curCluster['DATE'].iloc[x]
            total+= (b-a).total_seconds()
            count +=1
        x-=1
    if(total > 0 and count > 0):  
        #avgCluster.append(int(total/count))
        avgCluDict[curNumber]=int(total/count)
        
len(avgCluDict)#need create dictionary from cluster number and average

#append the new averages to dataframe
tempList = []
for row in df.iterrows():
    cluster = row[1]['Cluster']
    if(cluster in avgCluDict.keys()):
        tempList.append(avgCluDict[cluster])
df['DateAvg']=tempList


Use decision trees to predict cluster, when new data (latidude and logitude) comes in.

In [ ]:
# split data
X = df[['Latitude','Longitude']]
y = df['Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# create a regressor object
dtr = DecisionTreeRegressor(random_state = 0) 
dtr.fit(X_train, y_train)
dtr.score(X_test, y_test)
y_pred=dtr.predict(X_test)

In [ ]:
#get first prediction (cluster) & get acompaning average days
testCluster=y_pred[2]
sec = df[df.Cluster == int(testCluster)]['DateAvg'].iloc[0]

sec = timedelta(seconds=int(sec))
print(sec)

In [ ]:
#get last date of in that cluster & change to next earthquoke date
lastKnowDate=df[df.Cluster==int(testCluster)]['DATE'].iloc[-1] # 2016-12-20
new_datetime = lastKnowDate + sec
print("cluster : ",testCluster,"=>",new_datetime) # next earthquoke in the cluster 

In [ ]:
clu = df[df.Cluster==testCluster]
plot(clu)